<a href="https://colab.research.google.com/github/lpoggetto/fiap_liveness/blob/main/trabalho_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Limpando a pasta do colab para iniciar um novo projeto

In [1]:
# limpando a pasta
!rm -rf /content/*

### Instalando os pacotes utilizados no programa

In [2]:
 # instalando pacotes utilizados

!pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git
!pip install face_detection
!pip install mediapipe
!pip install dlib

print('pacotes instalados')

  Cloning https://github.com/hukkelas/DSFD-Pytorch-Inference.git to /tmp/pip-req-build-_6x9nd1p
  Running command git clone --filter=blob:none --quiet https://github.com/hukkelas/DSFD-Pytorch-Inference.git /tmp/pip-req-build-_6x9nd1p
  Resolved https://github.com/hukkelas/DSFD-Pytorch-Inference.git to commit dde9c7dd9cdc9254c2ca345222c86a8ecfa17f5b
  Preparing metadata (setup.py) ... done
pacotes instalados


### Importando os pacotes utilizados

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import bz2
import os
import face_detection
import mediapipe as mp
from scipy.spatial import distance as dist
import dlib
from PIL import Image
import warnings
from google.colab import output
warnings.filterwarnings('ignore')

#Exibição na mesma tela do Jupyter
%matplotlib inline
sns.set_style("whitegrid", {'axes.grid' : False})

# pacotes utilizados para tirar foto/videos
from IPython.display import display, Javascript, Image, HTML
from google.colab.output import eval_js
from base64 import b64decode
import threading

dlib.DLIB_USE_CUDA = True

### Criacao de pasta pasta para armazenamento dos videos

In [5]:
# criando diretorios para armazenar as informacoes necessarias
os.makedirs('videos', exist_ok=True) # validacao de video

### Importando os modelos utilizados no reconhecimento facial

In [6]:
# URL e nome dos arquivos
landmark_url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
landmark_file_name = "shape_predictor_68_face_landmarks.dat.bz2"
landmark_dat_file = "shape_predictor_68_face_landmarks.dat"

recognition_url = "http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2"
recognition_file_name = "dlib_face_recognition_resnet_model_v1.dat.bz2"
recognition_dat_file = "dlib_face_recognition_resnet_model_v1.dat"

# Criando diretório para salvar os modelos
os.makedirs('dlib_models', exist_ok=True)

# Apontando para o diretório
os.chdir('dlib_models')

# Função para baixar e extrair arquivos
def download_and_extract(url, compressed_file_name, extracted_file_name):
    # Baixando o arquivo se já não estiver baixado
    if not os.path.exists(compressed_file_name):
        os.system(f"wget {url}")

    # Extraindo o arquivo .dat se já não estiver extraído
    if not os.path.exists(extracted_file_name):
        with bz2.BZ2File(compressed_file_name, "rb") as f_in, open(extracted_file_name, "wb") as f_out:
            f_out.write(f_in.read())

        # Apagando o arquivo compactado
        os.remove(compressed_file_name)

# Baixando e extraindo o arquivo de landmarks
download_and_extract(landmark_url, landmark_file_name, landmark_dat_file)

# Baixando e extraindo o arquivo de reconhecimento facial
download_and_extract(recognition_url, recognition_file_name, recognition_dat_file)

# Voltando para o diretório original
os.chdir('..')

print('Modelos baixados e extraídos com sucesso!')

Modelos baixados e extraídos com sucesso!


### Capturando video para aplicar liveness

In [ ]:
# JavaScript para capturar video
html_code = """
<video id="video" width="640" height="480" autoplay></video>
<button id="startButton">Start Recording</button>
<button id="stopButton" disabled>Stop Recording</button>
<video id="playback" width="640" height="480" controls></video>
<a id="downloadLink" download="recorded-video.webm"></a>

<script>
    const video = document.querySelector('#video');
    const playback = document.querySelector('#playback');
    const downloadLink = document.querySelector('#downloadLink');
    const startButton = document.querySelector('#startButton');
    const stopButton = document.querySelector('#stopButton');

    navigator.mediaDevices.getUserMedia({ video: true }).then((stream) => {
        video.srcObject = stream;
        let mediaRecorder;
        const chunks = [];

        startButton.onclick = () => {
            mediaRecorder = new MediaRecorder(stream);
            mediaRecorder.start();
            startButton.disabled = true;
            stopButton.disabled = false;

            mediaRecorder.ondataavailable = (event) => chunks.push(event.data);
            mediaRecorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'video/webm' });
                playback.src = URL.createObjectURL(blob);
                downloadLink.href = playback.src;
                downloadLink.textContent = 'Download Video';
                startButton.disabled = false;
                stopButton.disabled = true;
            };
        };

        stopButton.onclick = () => mediaRecorder.stop();
    });
</script>
"""

# Display the HTML interface
display(HTML(html_code))

### Montando detector de faces

* Utilizando o algoritmo DSFDDetector para detectar uma face
* Caso seja identificado que o usuario piscou os olhos, a prova de vida sera considerada valida

In [10]:
def calculo_distancia_olhos(eye):
    # Calcula a distância euclidiana entre os pontos dos olhos
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    # Calcula a razão de aspecto do olho
    ear = (A + B) / (2.0 * C)
    return ear

def detector_faces_video(video_path, output_path, threshhold_ear=0.3):
    # Criando o detector de faces
    detector = face_detection.build_detector(
        "DSFDDetector",
        confidence_threshold=0.5,
        nms_iou_threshold=0.3
    )
    # utilizando algoritmo de 68 pontos faciais
    preditor_marcos_faciais = dlib.shape_predictor(
        '/content/dlib_models/shape_predictor_68_face_landmarks.dat'
        )

    # abrindo o arquivo de video
    video = cv2.VideoCapture(video_path)

    # propriedades do video
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'VP80')  # WebM codec
    out = cv2.VideoWriter(output_path, fourcc, 15, (width, height))

    # iniciando variaveis para a validacao de prova vida
    blink_detected = False

    fl_liveness = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # convertendo para RGB
        rgb_frame = frame[:, :, ::-1]

        # detectando  face
        detections = detector.detect(rgb_frame)

        # a partir do momento em que e identificado que o usuario piscou os olhos
        # a variavel de liveness redefine para 1
        if blink_detected == True:
            fl_liveness = 1

        # desenhando o retangulo na face
        for det in detections:
            x1, y1, x2, y2, score = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])


            # utilizando dlib para detectar a face
            dlib_rect = dlib.rectangle(x1,y1,x2,y2)

            # marcos faciais
            landmarks = preditor_marcos_faciais(rgb_frame, dlib_rect)

            # coordenadas dos olhos
            left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
            right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

            # calculando a distancia para ambos os olhos
            olho_esq = calculo_distancia_olhos(left_eye)
            olho_dir = calculo_distancia_olhos(right_eye)
            ear = (olho_esq + olho_dir) / 2

            # validacao de olhos piscando
            blink_detected = ear < threshhold_ear

            # checando se identifica o usuario piscando os olhos
            if fl_liveness == 1:
                # Caixa verde - Liveness OK
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, 'Liveness OK', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                # Caixa verde - Liveness Nao OK
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, 'Liveness nao OK', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # retorno do video
        out.write(frame)

    # Release resources
    video.release()
    out.release()
    print(f"Video salvo em: {output_path}")

# variaveis para uso da funcao
input_video_path = '/content/videos/recorded-video.webm'
output_video_path = '/content/videos/recorded-video_processado.webm'

# chamada da funcao criada anteriormente
detector_faces_video(input_video_path, output_video_path, threshhold_ear = 0.3)

Video salvo em: /content/videos/recorded-video_processado.webm
